In [ ]:
import snowflake.connector

# Establish a connection
con = snowflake.connector.connect(
    user='louspringer',
    account='CFCLGQP-HQB93436',  # Your Snowflake account identifier.
    private_key_file='/home/lou/snow-keys/rsa_key.p8',  # Ensure the path to your private key is correct.
    warehouse='PRD_ALV_WAREHOUSE',
    database='PRD_ALV',
    schema='QUERY_LINKEDIN'
)

In [1]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd

# Configure the engine with your Snowflake account details
connection_params = {
    'user': 'louspringer',  # Your typical username
    'private_key_file': '/home/lou/snow-keys/rsa_key.p8',  # Replace with your actual password
    'account': 'CFCLGQP-HQB93436',  # Specific to your Snowflake setup
    'warehouse': 'PRD_ALV_WAREHOUSE',
    'database': 'PRD_ALV',
    'schema': 'QUERY_LINKEDIN',
    'role': 'ALV_PROD_ROLE',  # Role configured for your project
}

engine = create_engine(URL(**connection_params))

# Query to select data from the table
query = "SELECT * FROM raw_jobs"

# Execute the query and fetch the data into a pandas DataFrame
df = pd.read_sql_query(query, engine)

/tmp/ipykernel_42926/1889655414.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, engine)


AttributeError: 'Engine' object has no attribute 'cursor'

In [8]:
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd
import os

# Load your private key from a file
with open("/home/lou/snow-keys/rsa_key.p8", "rb") as key_file:
    p_key= key_file.read()

connection_params = {
    'user': 'louspringer',
    'private_key_file': '/home/lou/snow-keys/rsa_key.p8',
    'account': 'CFCLGQP-HQB93436',
    'warehouse': 'PRD_ALV_WAREHOUSE',
    'database': 'PRD_ALV',
    'schema': 'QUERY_LINKEDIN',
    'role': 'ALV_PROD_ROLE',
}

engine = create_engine(URL(**connection_params))

# Query to select data from the table
query = "SELECT * FROM raw_jobs"

# Execute the query and fetch the data into a pandas DataFrame
df = pd.read_sql_query(query, engine)
print(df.head())

# Clean up by disposing the engine
engine.dispose()


/tmp/ipykernel_24307/2294182515.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, engine)


AttributeError: 'Engine' object has no attribute 'cursor'

In [9]:
# Test the engine's connection
try:
    with engine.connect() as connection:
        result = connection.execute("SELECT CURRENT_VERSION()")
        print(result.fetchone())
except Exception as e:
    print(f"Error testing the engine: {e}")

('8.18.0',)


/tmp/ipykernel_24307/1723696771.py:4: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  result = connection.execute("SELECT CURRENT_VERSION()")


In [6]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
import os

# Load your private key from a file
with open("/home/lou/snow-keys/rsa_key.p8", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,  # or provide a password if your key is encrypted
        backend=default_backend()
    )

# Convert the private key to the format Snowflake expects
p_key = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

connection_params = {
    'user': 'louspringer',
    'private_key': p_key,
    'account': 'CFCLGQP-HQB93436',
    'warehouse': 'PRD_ALV_WAREHOUSE',
    'database': 'PRD_ALV',
    'schema': 'QUERY_LINKEDIN',
    'role': 'ALV_PROD_ROLE',
}

engine = create_engine(URL(**connection_params))

try:
    with engine.connect() as connection:
        result = connection.execute("SELECT CURRENT_VERSION()")
        print(result.fetchone())
except Exception as e:
    print(f"Error testing the engine: {e}")

Error testing the engine: Expected bytes or RSAPrivateKey, got <class 'str'>


In [15]:
import os
os.environ['SQLALCHEMY_SILENCE_UBER_WARNING'] = '1'  # Optional: to suppress deprecation warnings temporarily

from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd

# Configure the engine with your Snowflake account details
connection_params = {
    'user': 'louspringer',
    'private_key_file': '/home/lou/snow-keys/rsa_key.p8',
    'account': 'CFCLGQP-HQB93436',
    'warehouse': 'PRD_ALV_WAREHOUSE',
    'database': 'PRD_ALV',
    'schema': 'QUERY_LINKEDIN',
    'role': 'ACCOUNTADMIN',
}

engine = create_engine(URL(**connection_params))

# Test the engine's connection
try:
    with engine.connect() as connection:
        result = connection.execute("SELECT CURRENT_VERSION()")
        print(result.fetchone())
except Exception as e:
    print(f"Error testing the engine: {e}")


('8.18.0',)


In [16]:
# Query to select data from the table
query = "SELECT * FROM raw_jobs"

# Execute the query and fetch the data into a pandas DataFrame
df = pd.read_sql_query(query, engine)
print(df.head())

# Clean up by disposing the engine
engine.dispose()

/tmp/ipykernel_24307/3253993156.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, engine)


AttributeError: 'Engine' object has no attribute 'cursor'

In [17]:
# Instead of using pd.read_sql_query, fetch data using raw SQLAlchemy connection
with engine.connect() as conn:
    result = conn.execute("SELECT * FROM raw_jobs")
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

print(df.head())

# Clean up
engine.dispose()

Empty DataFrame
Columns: [job_pk, request_pk, json_response, created_at]
Index: []


In [4]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
import os
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL
import pandas as pd

# Load your private key from a file and prepare it for use
with open("/home/lou/snow-keys/rsa_key.p8", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None,  # use the actual password if the key is encrypted
        backend=default_backend()
    )
private_key_bytes = private_key.private_bytes(
    encoding=serialization.Encoding.DER,
    format=serialization.PrivateFormat.PKCS8,
    encryption_algorithm=serialization.NoEncryption()
)

# Configure the SQLAlchemy engine with the private key bytes
engine = create_engine(URL(
        user='louspringer',
        account='CFCLGQP-HQB93436',
        warehouse='PRD_ALV_WAREHOUSE',
        database='PRD_ALV',
        schema='QUERY_LINKEDIN',
        role='ALV_PROD_ROLE',
    ),
    connect_args={
        'private_key': private_key_bytes,
    }
)

with engine.connect() as conn:
    df = pd.read_sql_query("SELECT * FROM raw_jobs", conn)
    
# # Query to select data from the table
# query = "SELECT * FROM raw_jobs"

# # Execute the query and fetch the data into a pandas DataFrame
# df = pd.read_sql_query(query, engine)

print(df.head())

# Clean up by disposing the engine
engine.dispose()


/tmp/ipykernel_42926/760331455.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM raw_jobs", conn)


AttributeError: 'Connection' object has no attribute 'cursor'

In [18]:
import pandas as pd
from sqlalchemy import create_engine
from snowflake.sqlalchemy import URL

# Ensure private_key_bytes is loaded properly as previously shown

engine = create_engine(URL(
    user='louspringer',
    account='CFCLGQP-HQB93436',
    warehouse='PRD_ALV_WAREHOUSE',
    database='PRD_ALV',
    schema='QUERY_LINKEDIN',
    role='ACCOUNTADMIN',
), connect_args={'private_key': private_key_bytes})

connection = engine.connect()

df = pd.read_sql("SELECT * FROM raw_jobs", con=connection.connection)

print(df.head())

Empty DataFrame
Columns: [JOB_PK, REQUEST_PK, JSON_RESPONSE, CREATED_AT]
Index: []


/tmp/ipykernel_42926/1471897548.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM raw_jobs", con=connection.connection)


In [13]:
connection = engine.connect() 

In [14]:
connection.connection
df = pd.read_sql("SELECT * FROM raw_jobs", con=connection.connection)

/tmp/ipykernel_42926/2900345364.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM raw_jobs", con=connection.connection)


In [6]:
import snowflake.connector
import pandas as pd

# Create a connection to Snowflake
ctx = snowflake.connector.connect(
    user='louspringer',
    private_key_file='/home/lou/snow-keys/rsa_key.p8',
    account='CFCLGQP-HQB93436',
    warehouse='PRD_ALV_WAREHOUSE',
    database='PRD_ALV',
    schema='QUERY_LINKEDIN',
    role='ALV_PROD_ROLE'
)

# Create a cursor object
cur = ctx.cursor()

# Execute a query
cur.execute("SELECT * FROM raw_jobs")

# Fetch the result into a pandas DataFrame
df = cur.fetch_pandas_all()

print(df.head())

# Close the cursor and connection
cur.close()
ctx.close()


Empty DataFrame
Columns: [JOB_PK, REQUEST_PK, JSON_RESPONSE, CREATED_AT]
Index: []


In [7]:
df

,JOB_PK,REQUEST_PK,JSON_RESPONSE,CREATED_AT
